In [9]:
import torch
import torchvision
import csv

import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.utils as vutils
import torchvision.transforms as transforms
import random

import pandas as pd
import matplotlib.pyplot as plt
import wandb

In [10]:
wandb.init(project="CPPNsquared")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gscsd. Use `wandb login --relogin` to force relogin


In [11]:
# set the device to run the model on
device = 'cuda'
# total training steps
num_steps = 500
# training batch size
batch_size = 8512
# how fast the learning rate is 
learn_rate = 0.001
# technique used in gradient descent to overcome local minima and oscilation
momentum = 0.9
# colour channels, 4 if alpha is present
num_channels = 3
# dimensions of input image
image_shape = (512, 644)

In [12]:
class CPPN(nn.Module):
    # defines initilizatoin class
    def __init__(self):
      # initiates itself
      super(CPPN, self).__init__()
      # fully connected layer 1, with 2 inputs and 16 outputs
      self.fc1 = nn.Linear(2, 64)
      # layer 2, 16 inputs, 32 outputs
      self.fc2 = nn.Linear(64, 64)
      self.fce1 = nn.Linear(64, 64)
      # layer 3, 32 inputs, output dependent on the number of colour channels
      self.fc3 = nn.Linear(64, num_channels)     
    
    # defines forward pass
    def forward(self, x):
        # forward through first layer
        x = self.fc1(x)
        # rectified linear activation function 
        x = F.relu(x)
        # layer 2
        x = self.fc2(x)
        # another reclinear function
        x = F.relu(x)
        x = self.fce1(x)
        x = F.relu(x)
        # layer 3
        x = self.fc3(x)
        # sigmoid activation function
        x = F.sigmoid(x)
        # returns the output of layer 3 after activation
        return x

In [13]:
cppn = CPPN()
cppn.laod_state_dict = torch.load('CPPN_model.pt')
print(cppn)
# feature maps trasfer learning 

CPPN(
  (fc1): Linear(in_features=2, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fce1): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=3, bias=True)
)


In [14]:
model_path = ('CPPN_model.pt')
original = torch.load(model_path).keys()
original

odict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fce1.weight', 'fce1.bias', 'fc3.weight', 'fc3.bias'])

In [15]:
# index, test = enumerate(cppn.named_parameters())
# print(test)

In [18]:
parameter_info = []
fc1 = []
full_coords = []
full_weights = []

def spatial_coords(array, layer):

    coords = []
    row1 = []
    row2 = []
    for i in range(array.shape[0]):
        row2.append([layer+1, i])
    for i in range(array.shape[1]):
        row1.append([layer, i])
            
    array = array.flatten("C").tolist()
    # print(array)
    for i in row2:
        for j in row1:
            temp = []
            temp.extend(j)
            temp.extend(i)
            coords.append((temp))
    coords_array = np.array(coords)
    # coords_array.astype(int)
    # output = np.column_stack((coords_array, array))

    # print(len(output))
    return coords, array
            
    # array.flatten()
    # return np.stack((array, coords))

index = 0
for name, param in cppn.named_parameters():
    
    # print(name)
    if name.endswith(".weight"):
        
        # print(index)
        print(param)
        temp_layer = param.detach().numpy() # need to learn more about gradients and why they are required
        # print(temp_layer)

        temp_coords, temp_weights = spatial_coords(temp_layer, index)
        full_coords.extend(temp_coords)
        full_weights.extend(temp_weights)

        index += 1
        # print(fc1.shape[0])


print(full_coords)
print(full_weights)
    
# layer1_coords, layer1_weights = spatial_coords(fce1, 2)

# print(layer1_arragned)
    
        

Parameter containing:
tensor([[-0.1679,  0.2120],
        [ 0.0147,  0.0323],
        [ 0.4526, -0.6684],
        [-0.4796, -0.1102],
        [-0.3828, -0.2710],
        [-0.3298, -0.6189],
        [ 0.0981,  0.0152],
        [-0.5832, -0.0679],
        [ 0.1369, -0.4728],
        [-0.0568,  0.6843],
        [ 0.5714,  0.1628],
        [ 0.5864, -0.3581],
        [-0.1493, -0.2427],
        [-0.3831, -0.6784],
        [-0.2321,  0.5456],
        [ 0.5424,  0.4706],
        [ 0.3499, -0.1468],
        [-0.0103, -0.3540],
        [ 0.0750,  0.1054],
        [ 0.0182,  0.2205],
        [-0.3504, -0.4873],
        [ 0.6405,  0.5447],
        [-0.4998,  0.6817],
        [ 0.3247,  0.4789],
        [ 0.6886,  0.6722],
        [-0.6001, -0.0012],
        [ 0.3187, -0.5136],
        [-0.0269, -0.5104],
        [ 0.4461,  0.1775],
        [ 0.2987,  0.4566],
        [-0.0463,  0.3141],
        [-0.6676, -0.6516],
        [ 0.0559, -0.0127],
        [-0.0430,  0.4486],
        [-0.4347,  0.2406]

In [19]:
class CPPN_squared(nn.Module):
    # defines initilizatoin class
    def __init__(self):
      # initiates itself
      super(CPPN_squared, self).__init__()
      # fully connected layer 1, with 2 inputs and 16 outputs
      self.fc1 = nn.Linear(4, 4)
      # layer 2, 16 inputs, 32 outputs
      # self.fc2 = nn.Linear(16, 16)
      # self.fce1 = nn.Linear(32, 64)
      # self.fce2 = nn.Linear(64,128)
      # layer 3, 32 inputs, output dependent on the number of colour channels
      self.fc3 = nn.Linear(4, 1)     
    
    # defines forward pass
    def forward(self, x):
        # forward through first layer
        x = self.fc1(x)
        # rectified linear activation function 
        x = F.relu(x)
        # layer 2
        # x = self.fc2(x)
        # # another reclinear function
        # x = F.relu(x)
        # x = self.fce1(x)
        # x = F.relu(x)
        # x = self.fce2(x)
        # x = F.relu(x)
        # # layer 3
        x = self.fc3(x)
        # sigmoid activation function
        x = F.sigmoid(x)

        # returns the output of layer 3 after activation
        return x

In [20]:
CPPN_squared = CPPN_squared()
wandb.watch(CPPN_squared)
CPPN_squared.to(device)
CPPN_squared.requires_grad_()

# optimiser = torch.optim.SGD(cppn.parameters(), lr=learn_rate, momentum=momentum)
optimiser = torch.optim.Adam(CPPN_squared.parameters(), lr=learn_rate)

criterion = nn.MSELoss(reduction='sum')

# criterion = nn.CrossEntropyLoss(reduction='sum')

In [21]:
def normalize_minus1_to_1(tensor):
    min_vals, _ = torch.min(tensor, dim=0)
    max_vals, _ = torch.max(tensor, dim=0)
    normalized_tensor = 2 * (tensor - min_vals) / (max_vals - min_vals) - 1
    return normalized_tensor

In [22]:
# print(len(layer1_weights))
all_xy_coordinates = torch.tensor(full_coords, device=device, dtype=torch.float32)
# print(all_xy_coordinates)

# mean_coords = all_xy_coordinates.mean(dim=0)
# std_coords = all_xy_coordinates.std(dim=0)

# Normalize the coordinates matrix along each column
normalized_coords_matrix = normalize_minus1_to_1(all_xy_coordinates)
print(normalized_coords_matrix)

all_pixel_values = torch.tensor(full_weights, device=device, dtype=torch.float32)
all_pixel_values = (all_pixel_values + 1)/2
# all_pixel_values = torch.unsqueeze(all_pixel_values, 0)
print(all_pixel_values)

tensor([[-1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -0.9683, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000, -0.9683],
        ...,
        [ 1.0000,  0.9365,  1.0000, -0.9365],
        [ 1.0000,  0.9683,  1.0000, -0.9365],
        [ 1.0000,  1.0000,  1.0000, -0.9365]], device='cuda:0')
tensor([0.4161, 0.6060, 0.5073,  ..., 0.5507, 0.4387, 0.4694], device='cuda:0')


In [23]:
from sklearn.model_selection import train_test_split

# train, test = train_test_split(normalized_coords_matrix, test_size= 0.2, random_state=42)
train_coords, val_coords, train_pixel_values, val_pixel_values = train_test_split(normalized_coords_matrix, all_pixel_values, test_size=0.2, random_state=42)

In [24]:
print(normalized_coords_matrix)

tensor([[-1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -0.9683, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000, -0.9683],
        ...,
        [ 1.0000,  0.9365,  1.0000, -0.9365],
        [ 1.0000,  0.9683,  1.0000, -0.9365],
        [ 1.0000,  1.0000,  1.0000, -0.9365]], device='cuda:0')


In [25]:
num_coords = all_xy_coordinates.shape[0]
coord_indexes = list(range(0, num_coords))
losses = []
img_list = []
running_loss = 0.0
best_loss = 100000

for i in range(num_steps):
    optimiser.zero_grad()
    cppn.zero_grad()

    # Sample a random batch of indexes from the list coord_indexes
    batch_indexes = torch.tensor(np.array(random.sample(coord_indexes, batch_size)))
    
    # Get batch of respective xy_coordiantes
    coordinates_batch = normalized_coords_matrix[batch_indexes]
    
    # And respective pixel values 
    pixel_values_batch = all_pixel_values[batch_indexes]
    pixel_values_batch = pixel_values_batch.unsqueeze(1)
    
    # Process data with model
    approx_pixel_values = CPPN_squared(coordinates_batch)
    
    # Calculate and track loss function
    loss = criterion(pixel_values_batch, approx_pixel_values)
    running_loss += loss.item()
    losses.append(loss.item())
    
    if i % 1 == 0:
        if running_loss < best_loss:
            best_loss = running_loss
            torch.save(CPPN_squared.state_dict(), 'CPPN2model.pt')
        print(f'step {i}, loss {running_loss/1000:.3f}')
        wandb.log({"loss": loss.item()})
        running_loss = 0.0
        with torch.no_grad():
            prediction = CPPN_squared(normalized_coords_matrix)
            print(prediction)
            # prediction  = torch.swapaxes(prediction, 0, 1)
            # prediction = torch.reshape(prediction, (num_channels, image_shape[1], image_shape[0]))
            # if not os.path.exists('training_ims'):
            #     os.makedirs('training_ims')
            # save_image(prediction, f'training_ims/im_{int(i/1000):06}.png')
            # img_list.append(prediction)

            
    #Update model
    loss.backward()
    optimiser.step()
    wandb.log({"final_weights": CPPN_squared.state_dict()})

step 0, loss 0.211
tensor([[0.6267],
        [0.6274],
        [0.6272],
        ...,
        [0.7048],
        [0.7063],
        [0.7078]], device='cuda:0')
step 1, loss 0.209
tensor([[0.6267],
        [0.6273],
        [0.6272],
        ...,
        [0.7031],
        [0.7046],
        [0.7061]], device='cuda:0')
step 2, loss 0.207
tensor([[0.6266],
        [0.6273],
        [0.6271],
        ...,
        [0.7015],
        [0.7030],
        [0.7045]], device='cuda:0')
step 3, loss 0.205
tensor([[0.6266],
        [0.6273],
        [0.6271],
        ...,
        [0.6998],
        [0.7013],
        [0.7028]], device='cuda:0')
step 4, loss 0.203
tensor([[0.6266],
        [0.6272],
        [0.6271],
        ...,
        [0.6982],
        [0.6997],
        [0.7011]], device='cuda:0')
step 5, loss 0.201
tensor([[0.6265],
        [0.6272],
        [0.6270],
        ...,
        [0.6965],
        [0.6980],
        [0.6995]], device='cuda:0')
step 6, loss 0.199
tensor([[0.6265],
        [0.6272

In [46]:
with torch.no_grad():
    torch.set_printoptions(profile="full")
    prediction = CPPN_squared(normalized_coords_matrix)
    prediction = prediction.flatten(0)
    # prediction = prediction * 2 - 1
    print(prediction)

tensor([0.5225, 0.5219, 0.5222, 0.5217, 0.5220, 0.5215, 0.5218, 0.5213, 0.5216,
        0.5210, 0.5214, 0.5208, 0.5212, 0.5206, 0.5209, 0.5204, 0.5207, 0.5202,
        0.5205, 0.5200, 0.5203, 0.5197, 0.5201, 0.5195, 0.5198, 0.5193, 0.5196,
        0.5191, 0.5194, 0.5189, 0.5192, 0.5186, 0.5190, 0.5184, 0.5188, 0.5182,
        0.5185, 0.5180, 0.5183, 0.5178, 0.5181, 0.5176, 0.5179, 0.5173, 0.5177,
        0.5171, 0.5175, 0.5169, 0.5172, 0.5167, 0.5170, 0.5165, 0.5168, 0.5162,
        0.5166, 0.5160, 0.5164, 0.5158, 0.5161, 0.5156, 0.5159, 0.5154, 0.5157,
        0.5152, 0.5155, 0.5149, 0.5153, 0.5147, 0.5151, 0.5145, 0.5148, 0.5143,
        0.5146, 0.5141, 0.5144, 0.5138, 0.5142, 0.5136, 0.5140, 0.5134, 0.5137,
        0.5132, 0.5135, 0.5130, 0.5133, 0.5128, 0.5131, 0.5125, 0.5129, 0.5123,
        0.5127, 0.5121, 0.5124, 0.5119, 0.5122, 0.5117, 0.5120, 0.5115, 0.5118,
        0.5112, 0.5116, 0.5110, 0.5113, 0.5108, 0.5111, 0.5106, 0.5109, 0.5104,
        0.5107, 0.5101, 0.5105, 0.5102, 